# Validating Cowbird user workspace on JupyterLab

This notebook checks that the user workspace is set up correctly by Cowbird and that the different related folders/files are accessible as expected on JupyterLab.

Note that this notebook is executed by the main notebook `notebooks-auth/test_cowbird_jupyter.ipynb`. It is executed using the [jhub-client](https://github.com/Quansight/jhub-client) package, although the package lacks certain features, such as displaying errors found in a notebook. This requires that the code found in this notebook displays errors with `print` instead of just raising an exception, since exceptions would not be detected by the package.

Another solution would have been to start this notebook using `nbval` and `docker exec` from the other notebook, but this would have required installing Docker on the `PAVICS-e2e-workflow-tests` image and mounting the Docker socket on the test container used in Jenkins, in order to be able to start and execute command on an external Docker container, outside of the test container. Mounting the Docker socket could bring some security issues, since it gives root access on the host, which made using [jhub-client](https://github.com/Quansight/jhub-client) preferable.

In [ ]:
import os
import shutil
import traceback
from pathlib import Path

def get_env(var_name):
    value = os.getenv(var_name)
    if not value:
        print("Missing environment variable `{}` required to run tests.".format(var_name))
    return value

test_user = get_env("TEST_COWBIRD_JUPYTERHUB_USERNAME")

# Path variables
jupyterhub_user_data_dir = get_env("JUPYTERHUB_USER_DATA_DIR")
notebook_dir = get_env("NOTEBOOK_DIR")
workspace_dir = get_env("WORKSPACE_DIR")

Check the user's `notebooks` directory, which should be a symlink to the user's Jupyterhub notebook directory.

In [ ]:
try:
    notebook_symlink_target_path = os.path.join(workspace_dir, "notebooks")
    assert os.path.islink(notebook_symlink_target_path), f"AssertionError: `{notebook_symlink_target_path}` is not a valid symlink."
    
    notebook_symlink_src_path = os.readlink(notebook_symlink_target_path)
    expected_symlink_src_path = os.path.join(jupyterhub_user_data_dir, test_user)
    assert notebook_symlink_src_path == expected_symlink_src_path, \
        f"AssertionError: Notebook symlink source path `{notebook_symlink_src_path}` does not correspond to the expected source path `{expected_symlink_src_path}`."
except Exception as exc:
    traceback.print_exc()

Check if the Geoserver datastore folder is setup correctly and if the user can create files in it

In [ ]:
user_datastore_path = os.path.join(workspace_dir, "shapefile_datastore")

try:
    assert os.path.isdir(user_datastore_path)
    test_filepath = f"{user_datastore_path}/test.txt"
    Path(test_filepath).touch()
    assert os.path.exists(test_filepath)
except Exception as exc:
    traceback.print_exc()

Check file permissions for Geoserver and WPS outputs files. `chown` / `chmod` commands should not be allowed to the user.

In [ ]:
public_wps_outputs_filepath = os.path.join(notebook_dir, "public/wps_outputs/weaver/public/3dc704a8-e1f6-4fe8-8bf0-153cf1e52c7a/output/test_public_file.txt")
user_wps_outputs_filepath = os.path.join(workspace_dir, "wps_outputs/weaver/test_user_file.txt")
rw_shapefile_path = os.path.join(user_datastore_path, "Espace_Vert_rw.shp")
ro_shapefile_path = os.path.join(user_datastore_path, "Espace_Vert_ro.shp")

OPERATION_NOT_PERMITTED_ERRNO = 1
READ_ONLY_FILE_SYSTEM_ERRNO = 30

try:
    for file, expected_errno, expected_write in [(public_wps_outputs_filepath, READ_ONLY_FILE_SYSTEM_ERRNO, False), 
                                                 (user_wps_outputs_filepath, OPERATION_NOT_PERMITTED_ERRNO, False),
                                                 (rw_shapefile_path, OPERATION_NOT_PERMITTED_ERRNO, True),
                                                 (ro_shapefile_path, OPERATION_NOT_PERMITTED_ERRNO, False)]:
        os.path.exists(file)

        # Try write operations
        write_successful = False
        try:
            with open(file, "a") as f:
               f.write("new text")
            write_successful = True
        except PermissionError as exc:
            # Expected error
            pass
        assert write_successful == expected_write

        # Try chmod operation
        chmod_error = False
        try:
            os.chmod(file, 0o777)
        except OSError as exc:
            # Expected error
            chmod_error = True
            try:
                assert exc.errno == expected_errno, f"Unexpected OSError : {exc}"
            except Exception as assert_exc:
                traceback.print_exc()
        assert chmod_error

        # Try chown operation
        chown_error = False
        try:
            os.chown(file, 0, 0)  # try to change ownership to default root uid/gid
        except OSError as exc:
            # Expected error
            chown_error = True
            try:
                assert exc.errno == expected_errno, f"Unexpected OSError : {exc}"
            except Exception as assert_exc:
                traceback.print_exc()
        assert chown_error
except Exception as exc:
    traceback.print_exc()
